In [10]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets.widgets as widgets
import numpy as np
from math import *
import time, sched
import board
import digitalio
from numpy import interp
from adafruit_servokit import ServoKit
from src import kinematics

In [12]:
kit = ServoKit(channels=16 , frequency=300)

# --- Servo Calibration ---
kit.servo[0].set_pulse_width_range(min_pulse=760, max_pulse = 2130)
kit.servo[1].set_pulse_width_range(min_pulse=810, max_pulse = 2180)
kit.servo[2].set_pulse_width_range(min_pulse=810, max_pulse = 2160)
kit.servo[3].set_pulse_width_range(min_pulse=870, max_pulse = 2225)
kit.servo[4].set_pulse_width_range(min_pulse=960, max_pulse = 2330)
kit.servo[5].set_pulse_width_range(min_pulse=890, max_pulse = 2160)
kit.servo[6].set_pulse_width_range(min_pulse=810, max_pulse = 2180)
kit.servo[7].set_pulse_width_range(min_pulse=780, max_pulse = 2140)
kit.servo[8].set_pulse_width_range(min_pulse=850, max_pulse = 2130)
kit.servo[9].set_pulse_width_range(min_pulse=800, max_pulse = 2170)
kit.servo[10].set_pulse_width_range(min_pulse=870, max_pulse = 2240)
kit.servo[11].set_pulse_width_range(min_pulse=860, max_pulse = 2140)

In [14]:
# Relay code to cut servo power if needed
# True = Power on
# False = Power off
relay = digitalio.DigitalInOut(board.D17)
relay.direction = digitalio.Direction.OUTPUT
relay.value = True


roll_slider = controller.axes[0]
yaw_slider = controller.axes[2]
pitch_slider = controller.axes[3]
def on_change(v):
    x = v['new']
    
roll_slider.observe(on_change, names = 'value')
yaw_slider.observe(on_change, names = 'value')
pitch_slider.observe(on_change, names = 'value')

In [15]:
# Arrays for leg position
# X-AXIS is forward and backwards
# Y-AXIS is Up and down
# Z-AXIS is left and right
# [x-axis, y-axis, z-axis]
# np.array([FL, FR, BL, BR])
Lp=np.array([[117,-sqrt(200)*10,88.5,1],[117,-sqrt(200)*10,-88.5,1],[-117,-sqrt(200)*10,88.5,1],[-117,-sqrt(200)*10,-88.5,1]])

In [16]:
# Function to write angles to servos
# Interpolate maps out the angle from -90 to 90 degrees to the servo configuriation of 0 to 180. 
# Some servos angles are inversed due to the installation of the servo.

def write_angles(angles):
    # FL
    if angles[0][0] >= -30 and angles[0][0] <= 30:
        kit.servo[0].angle = interp(angles[0][0], [-90, 90], [180, 0]) # Link 1

    if angles[0][1] >= -90 and angles[0][1] <= 30:
        kit.servo[1].angle = interp(angles[0][1], [-90, 90], [180, 0]) # Link 2

    if angles[0][2] >= -55 and angles[0][2] <= 60:
        kit.servo[2].angle = interp(angles[0][2], [-90, 90], [0, 180]) # Link 3

    # BL
    if angles[1][0] >= -30 and angles[1][0] <= 30:
        kit.servo[3].angle = interp(angles[1][0], [-90, 90], [0, 180]) # Link 1

    if angles[1][1] >= -90 and angles[1][1] <= 30:
        kit.servo[4].angle = interp(angles[1][1], [-90, 90], [180, 0]) # Link 2

    if angles[1][2] >= -55 and angles[1][2] <= 60:
        kit.servo[5].angle = interp(angles[1][2], [-90, 90], [0, 180]) # Link 3

    # BR
    if angles[2][0] >= -30 and angles[2][0] <= 30:
        kit.servo[6].angle = interp(angles[2][0], [-90, 90], [180, 0]) # Link 1

    if angles[2][1] >= -90 and angles[2][1] <= 30:
        kit.servo[7].angle = interp(angles[2][1], [-90, 90], [0, 180]) # Link 2

    if angles[2][2] >= -55 and angles[2][2] <= 60:
        kit.servo[8].angle = interp(angles[2][2], [-90, 90], [180, 0]) # Link 3

    # FR
    if angles[3][0] >= -30 and angles[3][0] <= 30:
        kit.servo[9].angle = interp(angles[3][0], [-90, 90], [0, 180]) # Link 1

    if angles[3][1] >= -90 and angles[3][1] <= 30:
        kit.servo[10].angle = interp(angles[3][1], [-90, 90], [0, 180]) # Link 2

    if angles[3][2] >= -55 and angles[3][2] <= 60:
        kit.servo[11].angle = interp(angles[3][2], [-90, 90], [180, 0]) # Link 3
        

In [17]:
controller = widgets.Controller(index = 0)
display(controller)

Controller()

In [19]:
r = controller.axes[0]
y = controller.axes[2]
p = controller.axes[3]
up = controller.buttons[12]
down = controller.buttons[13]
left = controller.buttons[14]
right = controller.buttons[15]
display(controller)

def f(r, y, p, up, down, left, right):
    roll = r*20*pi/180
    yaw = y*16*pi/180
    pitch = p*16*pi/180
    x_axis = 0
    y_axis = 0
    z_axis = 0
    if up == 1:
        y_axis = 20
    if down == 1:
        y_axis = -20
    if left == 1:
        z_axis = 15
    if right == 1:
        z_axis = -15
    angles = np.round(kinematics.solve(Lp,kinematics.bodyIK(roll,yaw,pitch,x_axis,y_axis,z_axis)), decimals=0)
    angles_s = np.array([[0, 0, 0],   # FL
                   [0, 0, 0],   # BL
                   [0, 0, 0],   # BR
                   [0, 0, 0]])  # FR
    write_angles(angles)
    print(angles)
    
out = widgets.interactive_output(f, {'r': r, 'y': y, 'p': p, 'up':up, 'down':down, 'left':left, 'right':right})
display(out)

Controller(axes=(Axis(value=0.0), Axis(value=0.0), Axis(value=0.0), Axis(value=0.0)), buttons=(Button(value=0.…

Output()

In [ ]:
relay.value = False

In [ ]:
beep = interp(250, [-90, 90], [180, 0])
print(beep)